In [ ]:
#%%capture
%load_ext autoreload
%autoreload 2

import numpy as np
np.set_printoptions(suppress=True, precision=3)


import matplotlib.pyplot as plt
from numpy.typing import NDArray

from select_deepc.base_controllers import UniformInputGenerator, RandomWalkInputGenerator 
from select_deepc.deepc_controller import *
from select_deepc.deepc_utils import *
from select_deepc.deepc_rocket_utils import *

from select_deepc.hankel_generation import *

from itertools import product

import pandas as pd
import pickle

from tqdm import tqdm

plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "serif",
        "font.sans-serif": "Times",
        "font.size": 10,
    }
)

PAPER_COL_WIDTH_IN = (6.75 - 0.25) / 2
PAPER_CONTENT_WIDTH_IN = 6.75
PAPER_COL_WIDTH_CM = PAPER_COL_WIDTH_IN * 2.54
PAPER_CONTENT_WIDTH_CM = PAPER_CONTENT_WIDTH_IN * 2.5

color_iso = "salmon"
color_l1 = "chocolate"
color_rand = "orchid"

## Data acquisition

We let the rocket fly using random inputs. In order to not crash, we use the existing pid controller to stabilize the system if we deviate too far from the initial position. After successfully acquiring the data, we will attempt land the rocket.

### Function definitions

In [ ]:
def run_data_collection_experiment(env, controller, seed=0):
  obs, info = env.reset(seed=seed)  # specify a random seed for consistency

  obs_init = obs[:2].copy()

  simulation_iteration = 0

  state_trajectory_data_acquisition = []
  input_trajectory_data_acquisition = []

  # simulate
  while True:
    controller_obs = obs.copy()

    action = controller.compute_action(controller_obs, np.zeros(8))

    next_obs, _, done, truncated, info = env.step(action)
    
    state_trajectory_data_acquisition.append(next_obs.copy())
    input_trajectory_data_acquisition.append(action)



    # check if simulation ended
    if done or truncated:
      print(f"info:\n{info}")
      break

    # update observation
    obs = next_obs
    simulation_iteration += 1

  env.close()  # video saved at this step

  # Drop last two columns as they are not relevant for the state propagation
  state_trajectory_data_acquisition = np.array(state_trajectory_data_acquisition)
  input_trajectory_data_acquisition = np.array(input_trajectory_data_acquisition)
  print(state_trajectory_data_acquisition.shape)

  return state_trajectory_data_acquisition, input_trajectory_data_acquisition

In [ ]:
def collect_random_data(mode):
    if mode not in ["iid", "random_walk"]:
        assert ValueError(f"Invalid value: {mode}. Options are: 'iid', 'random_walk")

    for experiment_idx in range(200):
        state_filename = f"data/reacher/dataset_{mode}/{experiment_idx}_random_data_state.csv"
        input_filename = f"data/reacher/dataset_{mode}/{experiment_idx}_random_data_input.csv"

        env = get_reacher_simulator(
            200,
            video_folder=os.path.join("video", "reacher", "data-collection"),
            video_title=f"experiment_{experiment_idx}"
        )
        if mode == "iid":
            state_traj, input_traj = run_data_collection_experiment(
                env, UniformInputGenerator([-0.1, -0.1], [0.1, 0.1], seed=experiment_idx), experiment_idx
            )

        if mode == "random_walk":
            state_traj, input_traj = run_data_collection_experiment(
                env, RandomWalkInputGenerator([-0.1, -0.1], [0.1, 0.1], [0, 0], seed=experiment_idx), experiment_idx
            )

        np.savetxt(state_filename, state_traj, delimiter=",")
        np.savetxt(input_filename, input_traj, delimiter=",")



### Data collection

In [ ]:
# collect_random_data("iid")
# collect_random_data("random_walk")

In [ ]:
iid_trajectories = load_data_from_folder("data/reacher/dataset_iid", "iid")
random_walk_trajectories = load_data_from_folder("data/reacher/dataset_random_walk", "random_walk")

def fix_dataset(dataset: TrajectoryDataSet):
    for traj in dataset.dataset:
        traj.state_trajectory[:, [4, 5]] += traj.state_trajectory[:, [8, 9]]
        # traj.state_trajectory[:, [0]] = np.arctan2(traj.state_trajectory[:, 2], traj.state_trajectory[:, 0])
        # traj.state_trajectory[:, [1]] = np.arctan2(traj.state_trajectory[:, 3], traj.state_trajectory[:, 1])
        # traj.state_trajectory = traj.state_trajectory[:, [0, 1, 4, 5, 6, 7]]
        traj.state_trajectory = traj.state_trajectory[:, [0, 1, 2, 3, 4, 5, 6, 7]]

for dataset in [iid_trajectories, random_walk_trajectories]:
    fix_dataset(dataset)

## DeePC

In [ ]:
def setup_DeePC(trajectory_data: TrajectoryDataSet, enable_measurement_constraint=False) -> DeePCControllerArgs:
    """"Set up the Data Driven Controller

    NOTE: Paramters in here can be adapted to tune the desired performance
    """
    p = 8 # sensor state size
    m = 2 # input size
    n = 75 # "estimated" system size
    T_past = 2
    T_fut = 15
    T_hankel= (m + 1) * (T_past + T_fut) + n - 1

    # x y w1 w2
    Q = np.diag([0, 0, 0, 0, 40000, 40000, 10, 10])
    R = np.diag([10, 10])
    g_regularization_1 = 10.000
    g_regularization_pi =  10000
    slack_cost = 5000000
    controller_costs = DeePCCost(Q, R, slack_cost, g_regularization_1, g_regularization_pi)

    A_u = None
    b_u = None
    A_y = None
    b_y = None

    A_u = np.array(
        [
            [1, 0],
            [-1, 0],
            [0, 1],
            [0, -1],
        ]
    )
    b_u = np.array(
        [
            0.5,
            0.5,
            0.5,
            0.5,
        ]
    )
    if enable_measurement_constraint:
        A_y = np.array(
            [
                [0, 0, 0, 0, 0, 1, 0, 0],
            ]
        )
        b_y = np.array(
            [
                [0.1],
            ]
        )

    controller_constraints = DeePCConstraints(A_u, b_u, A_y, b_y)

    return DeePCControllerArgs(
        trajectory_data,
        DeePCDims(
            T_past,
            T_fut,
            p,
            m,
        ),
        T_hankel,
        controller_costs,
        controller_constraints,
        [0, 0],
        False,
    )

class ReacherTargets:
    def __init__(self):
        num_steps = 4
        angles = np.linspace(3*np.pi/4, 2*np.pi, num_steps)
        np.random.seed(42)
        dists = np.random.uniform(0.04, 0.15, num_steps)

        self.targets = [[0.043, 0.092]]
        for angle, dist in zip(angles, dists):
            self.targets.append([dist*np.cos(angle), dist*np.sin(angle)])

        for target in self.targets:
            target[1] = np.minimum(target[1], 0.09)

class ReacherSetpointScheduler(SetPointScheduler):
    """Track a custom sequence of"""
    def __init__(self, env, deepc_dims: DeePCDims):
        super().__init__(env)
        self._dims = deepc_dims

        self._targets = ReacherTargets()
        self._sim_step = 1
        self._target_idx = 0

    def __call__(self, state, **kwargs):
        # target_pos = state[4:6]
        if (
            np.linalg.norm(
                state[4:6]
                + state[8:10]
                - np.array(self._targets.targets[self._target_idx])
            )
            < 0.005
        ):
            self._target_idx = (self._target_idx + 1) % len(self._targets.targets)
            print("updating setpoint")
            print(self._targets.targets[self._target_idx])

        self._sim_step += 1

        target_pos = self._targets.targets[self._target_idx]

        return [0, 0, 0, 0, target_pos[0], target_pos[1], 0, 0]

    def is_successful(self, state):
        return True


class DefaultReacherSetpoint(SetPointScheduler):
    """Track the setpoint provided by the env"""
    def __init__(self):
        pass

    def __call__(self, state, **kwargs):
        target_pos = state[4:6]
        return [0, 0, 0, 0, target_pos[0], target_pos[1], 0, 0]

    def is_successful(self, state):
        return True

In [ ]:
costs = {}
controller_args = setup_DeePC(
    iid_trajectories,
    enable_measurement_constraint=True,
)
solve_times = {}


for num_cols in [200]: #[100, 150, 200, 300, 1000]:
    env = get_reacher_simulator(100, video_title=f"select_deepc")

    selector = LkSelector()

    deepc = SelectDeePC(controller_args, selector, num_cols, 5)


    (
        x_traj_deepc,
        u_traj_deepc,
        x_traj_deepc_predictions,
        u_traj_deepc_predictions,
        cost,
        solve_time
    ) = run_reacher_simulator(
        env,
        deepc,
        ReacherSetpointScheduler(env, controller_args.deepc_dims),
        deepc_cost_accumulator=PerformanceAccumulatorWrapper(
            DeePCCostAccumulator(controller_args.controller_costs),
            IntegralSquareError(mask = np.array([0,0,0,0, 1, 1, 0, 0])),
            IntegralAbsoluteError(mask = np.array([0,0,0,0, 1, 1, 0, 0])),
        ),
    )
    print(cost)

    solve_times[num_cols] = solve_time

    plt.plot(x_traj_deepc[:,4] + x_traj_deepc[:,8], x_traj_deepc[:, 5] + x_traj_deepc[:,9])

    i = 0
    for arr in x_traj_deepc_predictions:
        i+= 1
        if i in [-1,]:
            continue

        plt.plot(arr[:, 4], arr[:, 5], "--", alpha=0.75)

    for sp in ReacherTargets().targets:
        plt.scatter(sp[0], sp[1], marker="x", c="black")

    plt.fill_between(np.linspace(-.25, .25), 0.1 * np.ones(50), 0.25 * np.ones(50), color="red", alpha=0.5, zorder=0)

    plt.show()

In [ ]:
with open("data/reacher/costs_l1.pkl", "rb") as f:
    costs = pickle.load(f)
with open("data/reacher/costs_rand.pkl", "rb") as f:
    costs_rand = pickle.load(f)
with open("data/reacher/costs_clustered_iso.pkl", "rb") as f:
    costs_iso = pickle.load(f)

In [ ]:
from matplotlib.lines import Line2D

blue = plt.rcParams["axes.prop_cycle"].by_key()["color"][0]
orange = plt.rcParams["axes.prop_cycle"].by_key()["color"][1]
green = plt.rcParams["axes.prop_cycle"].by_key()["color"][2]

fig, ax = plt.subplots(1,1, figsize=(PAPER_COL_WIDTH_IN,1.5))

ax.loglog(
    costs_rand.keys(),
    1e0*np.array([item["cost"] for item in costs_rand.values()]),
    linestyle="--",
    c=blue
)
ax.loglog(
    costs_rand.keys(),
    1e5*np.array([item["ISE"] for item in costs_rand.values()]),
    linestyle="--",
    c=green
)
ax.loglog(
    costs_rand.keys(),
    1e5 * np.array([item["IAE"] for item in costs_rand.values()]),
    linestyle="--",
    c=orange
)
ax.loglog(costs.keys(),  1e0* np.array([item["cost"] for item in costs.values()]),c=blue)
ax.loglog(costs.keys(), 1e5 * np.array([item["ISE"] for item in costs.values()]), c=green)
ax.loglog(costs.keys(), 1e5 * np.array([item["IAE"] for item in costs.values()]), c=orange)

ax.loglog(
    costs_iso.keys(), 1e0 * np.array([item["cost"] for item in costs_iso.values()]), ":", c=blue
)
ax.loglog(
    costs_iso.keys(), 1e5 * np.array([item["ISE"] for item in costs_iso.values()]), ":", c=green
)
ax.loglog(
    costs_iso.keys(), 1e5 * np.array([item["IAE"] for item in costs_iso.values()]), ":", c=orange
)

custom_lines = [
    Line2D([0],[0], color='black', linestyle="-"),
    Line2D([0],[0], color=blue, linestyle="-"),
    Line2D([0],[0], color='black',  linestyle="--"),
    Line2D([0],[0], color=orange, linestyle="-"),
    Line2D([0],[0], color='black',  linestyle=":"),
    Line2D([0],[0], color=green, linestyle="-"),
]
ax.set_xlabel("Number of Cols")
ax.set_ylabel("Cost")
ax.set_title("Cost in Reacher Simulation as\na Function of Hankel size")
ax.set_ylim(1e-1, None)
fig.legend(
    custom_lines,
    ["$L_1$", "Cost", "Random", "IAE", "Isomap", "ISE"],
    loc="lower left",
    bbox_to_anchor=(0.0, -.6),
    ncols=3,
)
# fig.legend(custom_lines_2, ["Cost", "IAE", "ISE"], loc="lower right")
fig.savefig("figures/reacher_costs_vs_rows.pdf", bbox_inches="tight")

In [ ]:
from matplotlib.lines import Line2D

blue = plt.rcParams["axes.prop_cycle"].by_key()["color"][0]
orange = plt.rcParams["axes.prop_cycle"].by_key()["color"][1]
green = plt.rcParams["axes.prop_cycle"].by_key()["color"][2]

fig, ax = plt.subplots(1, 1, figsize=(PAPER_COL_WIDTH_IN, 1.75))

ax.loglog(
    costs_rand.keys(),
    1e0 * np.array([item["cost"] for item in costs_rand.values()]),
    c=color_rand,
)

ax.loglog(
    costs.keys(),
    1e0 * np.array([item["cost"] for item in costs.values()]),
    c=color_l1,
)


ax.loglog(
    costs_iso.keys(),
    1e0 * np.array([item["cost"] for item in costs_iso.values()]),
    c=color_iso,
)

custom_lines = [
    Line2D([0], [0], color=color_rand),
    Line2D([0], [0], color=color_l1),
    Line2D([0], [0], color=color_iso),
]
ax.set_xlabel(r"$N_\textrm{cols}$")
ax.set_ylabel("Cost")
# ax.set_title("Closed-Loop Cost as\na Function of Subset Cardinality")
ax.set_title("Closed-Loop Cost for the Reacher")
ax.set_ylim(1e-1, None)
fig.legend(
    custom_lines,
    ["Random", "$L_1$", "Isomap"],
    loc="lower right",
    title=r"\textbf{Selection Method}",
    fontsize="small",
    ncols=3,
    bbox_to_anchor=(0.97,-0.2),
)
# fig.legend(custom_lines_2, ["Cost", "IAE", "ISE"], loc="lower right")
fig.tight_layout()
fig.savefig("figures/reacher_only_costs_vs_rows.pdf", bbox_inches="tight")

In [ ]:
fig, axs = plt.subplots(1,1, figsize=(3.5,3))

cl_data_sdeepc = np.loadtxt("data/reacher/cl_data/l1_selector/sdeepc_state_100.csv")
cl_data_sdeepc_pred = np.loadtxt("data/reacher/cl_data/l1_selector/sdeepc_predictions_100.csv")
cl_data_deepc = np.loadtxt("data/reacher/cl_data/deepc_state.csv")
cl_data_deepc_pred = np.loadtxt("data/reacher/cl_data/deepc_predictions.csv")
cl_data_streamdeepc = np.loadtxt("data/reacher/cl_data/windowed/deepc_state_100.csv")
cl_data_streamdeepc_pred = np.loadtxt("data/reacher/cl_data/windowed/deepc_predictions_100.csv")

blue = plt.rcParams["axes.prop_cycle"].by_key()["color"][0]
orange = plt.rcParams["axes.prop_cycle"].by_key()["color"][1]
green = plt.rcParams["axes.prop_cycle"].by_key()["color"][2]

axs.plot(cl_data_sdeepc[:,4] + cl_data_sdeepc[:,8], cl_data_sdeepc[:, 5] + cl_data_sdeepc[:,9], label="Select-DeePC", c=blue)
axs.plot(cl_data_deepc[:,4] + cl_data_deepc[:,8], cl_data_deepc[:, 5] + cl_data_deepc[:,9], zorder=10, label="Full Data DeePC", c=orange)
axs.plot(cl_data_streamdeepc[:,4] + cl_data_streamdeepc[:,8], cl_data_streamdeepc[:, 5] + cl_data_streamdeepc[:,9], label="Windowed DeePC", c=green)

for arr1, arr2 in zip(cl_data_sdeepc_pred[4::8],cl_data_sdeepc_pred[5::8]):
    axs.plot(arr1, arr2, "--", c=blue, alpha=0.5)

for arr1, arr2 in zip(cl_data_deepc_pred[4::8],cl_data_deepc_pred[5::8]):
    axs.plot(arr1, arr2, "--", c=orange, alpha=0.1)

for arr1, arr2 in zip(cl_data_streamdeepc_pred[4::8],cl_data_streamdeepc_pred[5::8]):
    axs.plot(arr1, arr2, "--", c=green, alpha=0.1)
    
axs.scatter([0.043], [0.092], marker="x", c="r", zorder=20, label="Setpoint")

axs.tick_params(
    axis='both',
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    left=False,
    right=False,
    labelbottom=False, # labels along the bottom edge are off)
    labelleft=False,
)

axs.plot([0,0.21], [0,0], "black", linewidth=10, alpha=.3)
axs.plot([0,0.105, 0.042], [0,0,0.096], "black", linewidth=10, alpha=.2)

# fig.legend(loc="lower left")
axs.set_xlabel("$x$")
axs.set_ylabel("$y$")
axs.set_title("Closed Loop Trajectory")
fig.tight_layout()
fig.savefig("figures/reacher_cl.pdf")

In [ ]:
fig, axs = plt.subplots(1,1, figsize=(PAPER_COL_WIDTH_IN, 2.75))

folder_constr = "good_constr"
folder_unconstr = "good_unconstr"
cl_data_sdeepc = np.loadtxt(f"data/reacher/multiple_setpoints/{folder_unconstr}/sdeepc_state_200.csv")
cl_data_sdeepc_pred = np.loadtxt(f"data/reacher/multiple_setpoints/{folder_unconstr}/sdeepc_predictions_200.csv")
cl_data_sdeepc_constr = np.loadtxt(f"data/reacher/multiple_setpoints/{folder_constr}/sdeepc_state_200.csv")
cl_data_sdeepc_pred_constr = np.loadtxt(f"data/reacher/multiple_setpoints/{folder_constr}/sdeepc_predictions_200.csv")

blue = plt.rcParams["axes.prop_cycle"].by_key()["color"][0]
orange = plt.rcParams["axes.prop_cycle"].by_key()["color"][1]
green = plt.rcParams["axes.prop_cycle"].by_key()["color"][2]

def plot_data(axis, data, label, color, clip_lentgth, **kwargs):
    x_data = data[:,4] + data[:,8]
    y_data = data[:,5] + data[:,9]
    x_data = x_data[:clip_lentgth]
    y_data = y_data[:clip_lentgth]
    axis.plot(x_data, y_data, label=label, c=color, **kwargs)

plot_data(axs, cl_data_sdeepc, "Unconstrained", blue, -3)
plot_data(
    axs, cl_data_sdeepc_constr, "Constrained", "midnightblue", -25, linestyle="-."
)

# axs.plot(, cl_data_sdeepc[:, 5] + cl_data_sdeepc[:,9], label="Select-DeePC", c=blue)
# axs.plot(cl_data_sdeepc_constr[:,4] + cl_data_sdeepc_constr[:,8], cl_data_sdeepc_constr[:, 5] + cl_data_sdeepc_constr[:,9], label="Select-DeePC", c=green)

# for idx, (arr1, arr2) in enumerate(zip(cl_data_sdeepc_pred[4::8],cl_data_sdeepc_pred[5::8])):
#     if idx < 17 or idx > 35:
#         continue
#     axs.plot(arr1, arr2, "--", c=blue, alpha=0.5)

# for idx, (arr1, arr2) in enumerate(zip(cl_data_sdeepc_pred_constr[4::8],cl_data_sdeepc_pred_constr[5::8])):
#     if idx < 14 or idx > 35:
#         continue
#     # print(arr2[arr2>0.1])
#     axs.plot(arr1, arr2, "--", c=green, alpha=0.5)


for idx, state in enumerate(cl_data_sdeepc_constr):
    theta_1 = np.arctan2(state[2], state[0])
    theta_2 = np.arctan2(state[3], state[1])
    # if idx in [0, 80, 221, 290]:
    if idx in [0, 10, 40, 50, 67]:
        axs.plot(
            0.1*np.array([0, np.cos(theta_1), np.cos(theta_1)+np.cos(theta_1+theta_2)]),
            0.1*np.array([0, np.sin(theta_1), np.sin(theta_1)+np.sin(theta_1+theta_2)]),
            c="black",
            alpha=0.75,
            linewidth=7,
            zorder=0,
        )


axs.tick_params(
    axis='both',
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    left=False,
    right=False,
    labelbottom=False, # labels along the bottom edge are off)
    labelleft=False,
)

# axs.plot([0,0.21], [0,0], "black", linewidth=10, alpha=.3)
for idx, sp in enumerate(ReacherTargets().targets):
    plt.scatter(sp[0], sp[1], marker="x", c="red")
    if idx == 3:
        break

# axs.scatter(0,0, marker="o", c="black")
axs.fill_between(
    np.linspace(-.25, .25), 0.1 * np.ones(50),
    0.25 * np.ones(50),
    color="indianred",
    hatch="\\",
    alpha=0.0,
    zorder=0,
)
# axs.fill_between(np.linspace(-.25, .25), -0.3 * np.ones(50), 0.1 * np.ones(50), color="green", alpha=0.25, zorder=0)
axs.plot(np.linspace(-.25, .25), 0.1*np.ones(50), color="indianred", linewidth=2, zorder=0)
axs.set_ylim(-.15, 0.2)
axs.set_xlim(-.15, 0.23)
axs.legend(loc="lower right")
# axs.plot([0,0.105, 0.042], [0,0,0.096], "black", linewidth=10, alpha=.2)

# fig.legend(loc="lower left")
axs.set_xlabel("$x$")
axs.set_ylabel("$y$")
axs.set_title("Select-DeePC Closed-Loop Trajectories")
fig.tight_layout()
fig.savefig("figures/reacher_cl_multiple_2.pdf", bbox_inches="tight")

In [ ]:
np.arctan2(cl_data_sdeepc[:,2], cl_data_sdeepc[:,0]).shape

In [ ]:
fig, axs = plt.subplots(1,1, figsize=(PAPER_COL_WIDTH_IN, 1.25))

# angle 1 select deepc
for idx, (arr1, arr2) in enumerate(zip(cl_data_sdeepc_pred[0::8,:], cl_data_sdeepc_pred[2::8,:])):
    axs.plot(
        5*idx + np.linspace(0,17,17),
        np.arctan2(arr2, arr1),
        "--",
        c=blue,
    )
axs.plot(
    np.linspace(0, cl_data_sdeepc.shape[0], cl_data_sdeepc.shape[0]),
    np.arctan2(cl_data_sdeepc[:,2], cl_data_sdeepc[:,0]),
    c=blue,
    label=r"$\theta_1$"
)

# angle 2 select deepc
for idx, (arr1, arr2) in enumerate(
    zip(cl_data_sdeepc_pred[1::8, :], cl_data_sdeepc_pred[3::8, :])
):
    axs.plot(5 * idx + np.linspace(0, 17, 17), np.arctan2(arr2, arr1), "--", c=orange)

axs.plot(
    np.linspace(0, cl_data_sdeepc.shape[0], cl_data_sdeepc.shape[0]),
    np.arctan2(cl_data_sdeepc[:, 3], cl_data_sdeepc[:, 1]),
    c=orange,
    label=r"$\theta_2$",
)

axs.legend()
axs.set_xlabel("Simulation time step")
axs.set_ylabel("Joint Angle [rad]")
axs.set_title("Reacher Joint Angle Evolution")
axs.legend(loc="lower left")

fig.savefig("figures/reacher_angles.pdf", bbox_inches="tight")

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.cm as cm

fig, axs = plt.subplots(1,2, figsize=(5,2.5), sharex=True, sharey=True, dpi=300)
# fig = plt.figure(figsize=(5,2.5))
# gs = fig.add_gridspec(
#     1, 3, width_ratios=[1, 1, 0.1]
# )  # The 3rd column is for the colorbar

# # Create subplots (ax1 and ax2)
# axs = []
# axs.append(fig.add_subplot(gs[0, 0]))
# axs.append(fig.add_subplot(gs[0, 1]))
# cbar_ax = fig.add_axes([0.85, 0.1, 0.05, 0.8])

H_u, H_y = HankelMatrixGenerator(2, 15).generate_hankel_matrices(iid_trajectories)
Y_p, Y_f = H_y[:2*(8),:], H_y[2*(8):,:]

H_z = np.vstack((H_u, Y_p, np.ones((1, H_u.shape[-1]))))

jac_fd = np.log(np.abs(Y_f @ np.linalg.pinv(H_z)))
# jac_fd = Y_f @ np.linalg.pinv(H_z)

# fig.colorbar(img)

cl_data_sdeepc_preds = np.loadtxt("data/reacher/cl_data/sdeepc_predictions.csv")
cl_data_sdeepc_input_preds = np.loadtxt("data/reacher/cl_data/sdeepc_input_preds.csv")

H_u, H_y = HankelMatrixGenerator(2, 15).generate_hankel_matrices(iid_trajectories)

sample_traj = np.zeros((2 * 17, 1)), cl_data_sdeepc_preds[0:8, :].reshape(-1,1)
idcs, _ = LkSelector()(sample_traj[0], sample_traj[1], H_u, H_y, None)
idcs_used = idcs[:200]
H_u, H_y = H_u[:,idcs_used], H_y[:,idcs_used]
Y_p, Y_f = H_y[: 2 * (8), :], H_y[2 * (8) :, :]

print(H_u.shape)
print(Y_p.shape, Y_f.shape)
H_z = np.vstack((H_u, Y_p, np.ones((1, H_u.shape[-1]))))
jac = np.log(np.abs(Y_f @ np.linalg.pinv(H_z)))
# jac = Y_f @ np.linalg.pinv(H_z)

norm = mcolors.Normalize(vmin=min(np.min(jac_fd), np.min(jac)), vmax=max(np.max(jac_fd), np.max(jac)))
img = axs[0].imshow(jac_fd, interpolation="nearest", aspect="auto", norm=norm, cmap=cm.viridis)
img = axs[1].imshow(jac, interpolation="nearest", aspect="auto", norm=norm, cmap=cm.viridis)
cbar_width = 0.85
cax = fig.add_axes([1, 0.07, 0.025, 0.79])
fig.colorbar(
    img,
    cax=cax,
    # ax=axs,
    # shrink=0.5,
    label=r"log(abs($Y_\textrm{f}H_z$))")
axs[0].set_title("Full Data Jacobian")
axs[1].set_title("Selected Data Jacobian")
axs[0].set_axis_off()
axs[1].set_axis_off()
fig.tight_layout()
fig.savefig("figures/reacher_jacs.pdf", bbox_inches="tight")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
data = {
    150: [0.07556, 0.1721],
    200: [0.078, 0.231],
    250: [0.0735, 0.2929],
    300: [0.062, 0.321],
    500: [0.05, 0.858],
    1000: [0.09232, 2.972],
}
data_l1 = {
    150: [0.0044, 0.1364],
    200: [0.0047, 0.201],
    250: [0.0049, 0.2722],
    300: [0.0045, 0.313],
    500: [0.004, 0.879],
    1000: [0.003, 3.385],
}
x = []
y_1 = []
y_2 = []
for key, vals in data.items():
    x.append(key)
    y_1.append(vals[0])
    y_2.append(vals[1])

selcolor = "darkgoldenrod"
qpcolor = "wheat"
fig, ax = plt.subplots(1,1,figsize=(PAPER_COL_WIDTH_IN,1.75))
ax.bar(x, y_1, width=50, label="Selection", color=selcolor)
ax.bar(x, y_2, bottom=y_1, width=50, label="QP Solve", color=qpcolor)

ax.plot(np.linspace(150, 1000), 0.075*np.ones(50), "--", label="$\\mathcal{O}(1)$", color=selcolor)
lspc = np.linspace(150, 1000)
ax.plot(lspc, 2.8e-6*lspc**2 +0.18, "--", label="$\\mathcal{O}(n^2)$", color=qpcolor)
ax.set_xlabel(r"$N_\textrm{cols}$")
ax.set_ylabel("Solve Time [s]")
handles, labels = plt.gca().get_legend_handles_labels()
order = [2,3,0,1]
ax.set_title("Computation Times of Select-DeePC")
ax.set_yscale("log")
ax.set_ylim(0.04, None)
fig.legend(
    [handles[idx] for idx in order],
    [labels[idx] for idx in order],
    fontsize="small",
    ncols=4,
    bbox_to_anchor=(1.0, 0.08),
    handlelength=1.25,
    )

fig.tight_layout()
fig.savefig("figures/deepc_solve_times.pdf", bbox_inches="tight")